In [174]:
import numpy as np
import pandas as pd
import netCDF4
import glob
import os

In [175]:
os.chdir('/Volumes/My Passport/eem20/')

In [176]:
# sector power
sector_power = pd.read_csv('data_turbines/windpower_task2.csv')

#Only select the last 61 days
drop_indices = np.arange(0,10200).tolist()
sector_power = sector_power.drop(index=drop_indices).reset_index()


In [177]:
# sort out the global time stamp list for the weather files

listoffiles = (glob.glob("Round 2/Comp data/*.nc"))
time_stamps = []
for file in listoffiles:
    weather = netCDF4.Dataset(file, 'r')
    units = weather.variables['time'].units
    Timestamp = netCDF4.num2date(weather.variables['time'][:], units=units)
    time_stamps += Timestamp.tolist()

# sort into time-ordered array
time_stamps.sort()

In [178]:
#Load the targets

targets_SE1 = sector_power['SE1'].values
targets_SE2 = sector_power['SE2'].values
targets_SE3 = sector_power['SE3'].values
targets_SE4 = sector_power['SE4'].values

In [181]:
#Load the predictions

predictions_SE1 = np.load('round2_percentiles/percentiles_SE1.npy')
predictions_SE2 = np.load('round2_percentiles/percentiles_SE2.npy')
predictions_SE3 = np.load('round2_percentiles/percentiles_SE3.npy')
predictions_SE4 = np.load('round2_percentiles/percentiles_SE4.npy')

In [182]:
#List of quantiles
tau = np.arange(0.1, 1, 0.1)

#Define ReLU function
def ReLU(x):
    return max(0,x)

#Calculate the averaged pinball loss function per area, and average over all the timestamps

Loss_SE1 = 0
for i in range(len(time_stamps)):
    for j in range(len(tau)):
        a = tau[j] * ReLU(targets_SE1[i] - predictions_SE1[i,j]) + (1-tau[j])*ReLU(predictions_SE1[i,j] - targets_SE1[i])
        Loss_SE1 += a
Loss_SE1 = Loss_SE1/(9*len(time_stamps))

Loss_SE2 = 0
for i in range(len(time_stamps)):
    for j in range(len(tau)):
        a = tau[j] * ReLU(targets_SE2[i] - predictions_SE2[i,j]) + (1-tau[j])*ReLU(predictions_SE2[i,j] - targets_SE2[i])
        Loss_SE2 += a
Loss_SE2 = Loss_SE2/(9*len(time_stamps))

Loss_SE3 = 0
for i in range(len(time_stamps)):
    for j in range(len(tau)):
        a = tau[j] * ReLU(targets_SE3[i] - predictions_SE3[i,j]) + (1-tau[j])*ReLU(predictions_SE3[i,j] - targets_SE3[i])
        Loss_SE3 += a
Loss_SE3 = Loss_SE3/(9*len(time_stamps))

Loss_SE4 = 0
for i in range(len(time_stamps)):
    for j in range(len(tau)):
        a = tau[j] * ReLU(targets_SE4[i] - predictions_SE4[i,j]) + (1-tau[j])*ReLU(predictions_SE4[i,j] - targets_SE4[i])
        Loss_SE4 += a
Loss_SE4 = Loss_SE4/(9*len(time_stamps))


#team score of each submission is calculated as an average over all areas
Team_score = (Loss_SE1 + Loss_SE2 + Loss_SE3 + Loss_SE4)/4
Team_score

76.34018527518245